In [1]:
import numpy;
# import matplotlib.pyplot as plt;

from tensorflow.keras.initializers import TruncatedNormal;

from tqdm import tqdm;

import numpy as np
import random

2024-05-27 13:13:02.098916: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Training data
feature = numpy.array(
    [
        [0.08, 0.72], [0.26, 0.58], [0.45, 0.15], 
        [0.60, 0.30], [0.10, 1.0], [0.35, 0.95], 
        [0.70, 0.65], [0.92, 0.45]
    ]);

label = numpy.array([1, 1, 1, 1, -1, -1, -1, -1]);

In [17]:
class AdalineGA:

    # Hyperparameters
    def __init__(self, feature, label):
        self.feature = feature;
        self.label = label;
        self.population_size = 50;
        self.generations = 200;
        self.mutation_rate = 1e-2;
        self.crossover_rate = 7e-1;

        initializer = TruncatedNormal(mean = 0.5, stddev = 0.02)(shape = (1, 3)).numpy();
        
        # Initialize population
        self.population = [];

        for _ in range(self.population_size):
            individual = {
                'weight_x1': initializer[0][0],
                'weight_x2': initializer[0][1],
                'bias': initializer[0][2]
            };
        
            self.population.append(individual);

    def select_population(self, population):
        sorted_population = sorted(population, key = self.forward, reverse = True);
        return sorted_population[:int(0.2 * self.population_size)];  # Select top 20%

    def crossover(self, parent1, parent2):
        if random.random() < self.crossover_rate:
            crossover_point = random.randint(0, 2);
            child1 = parent1.copy();
            child2 = parent2.copy();
            if crossover_point == 0:
                child1['weight_x1'], child2['weight_x1'] = child2['weight_x1'], child1['weight_x1'];
            elif crossover_point == 1:
                child1['weight_x2'], child2['weight_x2'] = child2['weight_x2'], child1['weight_x2'];
            else:
                child1['bias'], child2['bias'] = child2['bias'], child1['bias'];
            return child1, child2;
        else:
            return parent1, parent2;

    def mutate(self, individual):
        if random.random() < self.mutation_rate:
            individual['weight_x1'] += np.random.normal();
        if random.random() < self.mutation_rate:
            individual['weight_x2'] += np.random.normal();
        if random.random() < self.mutation_rate:
            individual['bias'] += np.random.normal();
        
        return individual;

    def forward(self, individual):
        predictions = [];
        for i in self.feature:
            prediction = (individual['weight_x1'] * i[0]) + (individual['weight_x2'] * i[1]) + individual['bias']
            predictions.append(prediction)
        mse = np.mean((self.label - predictions) ** 2);
        return -mse;

    # Get the best produced individual based on loss
    def get_best_individual(self):
        
        result = [];

        for i in self.population:
            loss = self.forward(i);
            result.append({
                "weight_x1": i["weight_x1"],
                "weight_x2": i["weight_x2"],
                "bias": i["bias"],
                "loss": loss,
            })

        best_individual = max(result, key = lambda x: x["loss"]);

        return best_individual["weight_x1"], best_individual["weight_x2"], best_individual["bias"], best_individual["loss"];

    def train(self):

        for generation in range(self.generations):
            
            print(f"Generation [{generation + 1} / {self.generations}] -", end = " ");

            new_population = [];

            selected_individuals = self.select_population(self.population);

            for i in tqdm(range(self.population_size)):

                parent1 = random.choice(selected_individuals);
                parent2 = random.choice(selected_individuals);

                child1, child2 = self.crossover(parent1, parent2);
                
                new_population.append(self.mutate(child1));
                new_population.append(self.mutate(child2));
                

            self.population = new_population;

            best_weight_x1, best_weight_x2, best_bias, loss = self.get_best_individual();

            print(f"Best weights of generation [{generation + 1}] - weight_x1: {best_weight_x1} - weight_x2: {best_weight_x2} - bias: {best_bias} - loss: {loss}\n\n");

In [18]:
model = AdalineGA(feature, label);
model.train();

Generation [1 / 200] - 

100%|██████████| 50/50 [00:00<00:00, 53308.39it/s]


{'weight_x1': 0.524304, 'weight_x2': 0.49561256, 'bias': 0.48108166, 'loss': -2.279758508118605}
Best weights of generation [1] - weight_x1: 0.5243039727210999 - weight_x2: 0.49561256170272827 - bias: 0.48108166456222534 - loss: -2.279758508118605


Generation [2 / 200] - 

100%|██████████| 50/50 [00:00<00:00, 75086.00it/s]


{'weight_x1': 0.524304, 'weight_x2': 0.49561256, 'bias': 0.48108166, 'loss': -2.279758508118605}
Best weights of generation [2] - weight_x1: 0.5243039727210999 - weight_x2: 0.49561256170272827 - bias: 0.48108166456222534 - loss: -2.279758508118605


Generation [3 / 200] - 

...

{'weight_x1': -2.443299806292322, 'weight_x2': -2.681818121403576, 'bias': 2.634736934955175, 'loss': -0.19923246333769726}
Best weights of generation [198] - weight_x1: -2.443299806292322 - weight_x2: -2.681818121403576 - bias: 2.634736934955175 - loss: -0.19923246333769726


Generation [199 / 200] - 

100%|██████████| 50/50 [00:00<00:00, 45919.68it/s]


{'weight_x1': -2.443299806292322, 'weight_x2': -2.681818121403576, 'bias': 2.634736934955175, 'loss': -0.19923246333769726}
Best weights of generation [199] - weight_x1: -2.443299806292322 - weight_x2: -2.681818121403576 - bias: 2.634736934955175 - loss: -0.19923246333769726


Generation [200 / 200] - 

100%|██████████| 50/50 [00:00<00:00, 82727.89it/s]


{'weight_x1': -2.443299806292322, 'weight_x2': -2.681818121403576, 'bias': 2.634736934955175, 'loss': -0.19923246333769726}
Best weights of generation [200] - weight_x1: -2.443299806292322 - weight_x2: -2.681818121403576 - bias: 2.634736934955175 - loss: -0.19923246333769726


